In [98]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm
from time import sleep
import random
import pandas as pd
import plotly.graph_objects as go
import json
from itertools import tee, islice
#import dash
from dash import Dash, dcc, html, Input, Output, dash_table
# import dash_core_components as dcc 
# import dash_html_components as html

In [210]:
pd.set_option('display.max_rows', 50)  
#печать всех строх, аналогично max_columns

# You can also reset an option back to its default value like this:
# pd.reset_option('display.max_rows')
# And reset all of them back:
# pd.reset_option('all')

In [100]:
# вывести долю еще и от общего количества

## Cборка

In [101]:
folder_path = 'ooh_data'

In [102]:
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

In [103]:
dataframes = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]

In [104]:
dfs = pd.concat(dataframes, ignore_index=True)
dfs.shape

(7531594, 4)

In [105]:
dfs.columns = ['order_id', 'status', 'source', 'status_time']

In [106]:
dfs['status_time'] = pd.to_datetime(dfs['status_time'])

In [107]:
dfs.status_time = dfs.status_time.dt.tz_localize(None)

In [108]:
# Прибавляем 3 часа
dfs.status_time += pd.Timedelta(hours=3)

In [109]:
dfs.shape

(7531594, 4)

In [110]:
dfs.drop_duplicates().shape

(7531594, 4)

In [111]:
#dfs.groupby(dfs['status_time'].dt.date).count()

In [112]:
dfsp = pd.read_csv('https://redash.puls.ru/api/queries/8589/results.csv?api_key=yBnPjPTW9feg3qVwlkvDJYL8EpT0uZfzwGT5Kv7o')

In [113]:
dfsp['status_time'] = pd.to_datetime(dfsp['status_time'])
# Прибавляем 3 часа
dfsp.status_time += pd.Timedelta(hours=3)

In [114]:
dfs = pd.concat([dfs, dfsp])

In [115]:
dfs

,order_id,status,source,status_time
0,6331113,4,NaN,2023-10-31 11:27:34.989
1,6228721,7,NaN,2023-10-31 11:27:35.419
2,6163311,7,NaN,2023-10-31 11:27:35.937
3,6226536,7,NaN,2023-10-31 11:27:36.050
4,6195567,7,NaN,2023-10-31 11:27:36.509
...,...,...,...,...
179301,1587399,4,NaN,2023-11-28 20:44:30.000
179302,1587465,1,NaN,2023-11-28 20:47:53.000
179303,1586530,5,NaN,2023-11-28 20:50:07.000
179304,1587473,2,NaN,2023-11-28 20:56:02.000


In [116]:
# номера заказов из еком
dfo = pd.read_csv('https://redash.puls.ru/api/queries/8525/results.csv?api_key=cLN1dxPKexPyj7an8hXyaOZHY3zv2l4uwTPPZS42')

In [117]:
dfop = pd.read_csv('https://redash.puls.ru/api/queries/8537/results.csv?api_key=QgDv1XehCGYgkDIWmiyP1J6sPSUsXVw8WwYjoRGy')

In [118]:
dfo = pd.concat([dfo, dfop])

In [119]:
dfo['created'] = pd.to_datetime(dfo['created'])

In [120]:
dfo = dfo[dfo['created'].dt.month == 11]

In [23]:
#dfo[(dfo.inn.isna()) & (dfo.brand == 'Планета Здоровья' )]

In [121]:
dfo.marketplace_number = dfo.marketplace_number.astype('str')

In [122]:
dfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596950 entries, 0 to 20614
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  596950 non-null  int64         
 1   guid                596611 non-null  object        
 2   marketplace_number  596950 non-null  object        
 3   mm                  596950 non-null  object        
 4   rk                  596950 non-null  object        
 5   brand               551495 non-null  object        
 6   name                596950 non-null  object        
 7   inn                 554176 non-null  float64       
 8   created             596950 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 45.5+ MB


In [123]:
#dfo['inn'] = dfo['inn'].fillna(0).astype(int)
dfo['innst'] = dfo['inn'].astype(str)

In [125]:
inn_erka = pd.read_excel('inn_erka.xlsx')

In [126]:
inn_pz = pd.read_excel('inn_pz.xlsx')

In [127]:
dfo_erka = dfo[dfo.inn.isin(inn_erka.inn)]

In [128]:
dfo_pz = dfo[dfo.inn.isin(inn_pz.inn) | (dfo.brand == 'Планета Здоровья' )]

In [129]:
dfo_erka.id.nunique()

30831

In [130]:
dfo_pz.id.nunique()

134144

In [131]:
dfo_erka.brand.unique()

array(['Озерки', 'Супераптека', 'Самсон Фарма', 'Доктор Столетов',
       'Аптека.ру', 'МосАптека', 'Первая Помощь', 'Эркафарм', 'Фармация',
       'Доктор Столетов (старый)', 'Озерки у дома', 'Радуга', nan],
      dtype=object)

In [132]:
dfo_pz.brand.unique()

array(['Планета Здоровья', 'ПОЛИФАРМ', 'Реон', 'Планета Pдоровья',
       'Авиценна', 'ПРАЦЕ', nan, 'Карсилс', 'Грант', 'Юнити', 'Альфа',
       'Ронна (Планета здоровья/Опека СПб)', 'Аптека №59', 'Кант',
       'Плутос', 'Планета здоровья'], dtype=object)

In [133]:
dfs71 = dfs[dfs.status == 71]
dfs9 = dfs[dfs.status == 9]

In [134]:
# Заказы от Эркафарм
erka = pd.read_excel('Заказы пульса, Эркафарм.xlsx')

In [135]:
erka = erka.iloc[:, [0, 4, 5]]
erka.columns = ['er_order_id', 'er_status', 'er_status_time']

In [136]:
erka['er_status_time'] = pd.to_datetime(erka['er_status_time'], dayfirst=True)

In [137]:
erka71 = erka[erka.er_status == 'Собрали заказ / Оттправили Ready']

In [138]:
erka9 = erka[erka.er_status == 'Продан']

In [139]:
# Заказы от Планеты здоровья
pz = pd.read_excel('Заказы пульса за ноябрь, Планета Здоровья.xlsx')

In [140]:
pz = pz.iloc[:, [1, 2, 3]]

In [141]:
pz.columns = ['pz_order_guid', 'pz_status', 'pz_status_time']

In [142]:
pz71 = pz[pz.pz_status == 'ready']

In [143]:
pz9 = pz[pz.pz_status == 'sold']

In [221]:
vyg = dfo_pz71[(dfo_pz71.mm == 'В аптеке') & (~dfo_pz71.order_id.isna())]      \
            .loc[:, ['marketplace_number', 'guid', 'rk', 'created', 'innst', 'pz_order_guid']]   \
            .drop_duplicates()

In [222]:
vyg.to_excel('output.xlsx', index=False)

In [218]:
# aa = dfo_pz71[(dfo_pz71.mm == 'В аптеке') & (~dfo_pz71.order_id.isna())] \
#     .loc[:, ['marketplace_number', 'rk', 'created', 'innst', 'pz_order_guid']]
# aa[~aa.pz_order_guid.isna()].pz_order_guid.nunique()

In [219]:
# dfo_pz71[(dfo_pz71.mm == 'В аптеке') & (~dfo_pz71.order_id.isna())].order_id.nunique()

## Эркафарм 71

In [144]:
dfo_erka71 = dfo_erka.merge(dfs71, how='left', left_on='id', right_on='order_id')

In [145]:
dfo_erka71.shape

(30881, 14)

In [204]:
# есть 50 дублей, очевидно из-за того, что статусы по 2 раза приходят. на общие цифры не повлияет, все равно уникальные считаю

In [147]:
dfo_erka71 = dfo_erka71.merge(erka71, how='left', left_on='marketplace_number', right_on='er_order_id')

In [148]:
# есть еще 10 дублей, хз почему

In [149]:
dfo_erka71_mm = dfo_erka71.groupby('mm', as_index=False) \
                        .agg({'id': 'nunique', 
                           'order_id': 'nunique', 
                           'er_order_id': 'nunique'})

In [150]:
dfo_erka71_mm['percent'] = round(dfo_erka71_mm.order_id / dfo_erka71_mm.er_order_id * 100, 0)

In [151]:
dfo_erka71_mm

,mm,id,order_id,er_order_id,percent
0,АптекаФорте,782,754,752,100.0
1,Все аптеки,743,668,720,93.0
2,ОК Аптека,22,21,21,100.0
3,Озон RFBS,6728,5917,6550,90.0
4,Сбермегамаркет,894,831,876,95.0
5,Ютека,20783,18650,20305,92.0
6,ЯндексDBS,879,879,867,101.0


In [152]:
dfo_erka71_rk = dfo_erka71.groupby('rk', as_index=False).agg({'id': 'nunique', 
                                               'order_id': 'nunique', 
                                               'er_order_id': 'nunique'})
dfo_erka71_rk['percent'] = round(dfo_erka71_rk.order_id / dfo_erka71_rk.er_order_id * 100, 0)

In [153]:
dfo_erka71_rk

,rk,id,order_id,er_order_id,percent
0,"ООО ""ПУЛЬС Брянск""",473,470,462,102.0
1,"ООО ""ПУЛЬС Волгоград""",1376,1366,1373,99.0
2,"ООО ""ПУЛЬС Воронеж""",109,106,107,99.0
3,"ООО ""ПУЛЬС Екатеринбург""",123,123,122,101.0
4,"ООО ""ПУЛЬС Казань""",149,149,148,101.0
5,"ООО ""ПУЛЬС Краснодар""",632,626,623,100.0
6,"ООО ""ПУЛЬС Красноярск""",88,86,84,102.0
7,"ООО ""ПУЛЬС Новосибирск""",1539,1512,1489,102.0
8,"ООО ""ПУЛЬС СПб""",9765,9566,9608,100.0
9,"ООО ""ПУЛЬС Самара""",168,168,168,100.0


In [154]:
dfo_erka71_dd = dfo_erka71.groupby(dfo_erka71.created.dt.date) \
                        .agg({'id': 'nunique', 
                           'order_id': 'nunique', 
                           'er_order_id': 'nunique'})
dfo_erka71_dd['percent'] = round(dfo_erka71_dd.order_id / dfo_erka71_dd.er_order_id * 100, 0)

In [155]:
dfo_erka71_dd

,id,order_id,er_order_id,percent
created,,,,
2023-11-01,534,528,533,99.0
2023-11-02,833,811,828,98.0
2023-11-03,1113,1058,1111,95.0
2023-11-04,805,663,803,83.0
2023-11-05,863,731,861,85.0
2023-11-06,1094,886,1094,81.0
2023-11-07,1284,1196,1279,94.0
2023-11-08,1250,1209,1248,97.0
2023-11-09,1244,1103,1242,89.0


## Эркафарм 9

In [156]:
dfo_erka9 = dfo_erka.merge(dfs9, how='left', left_on='id', right_on='order_id')

In [157]:
dfo_erka9.shape

(30845, 14)

In [158]:
# есть 50 дублей, очевидно из-за того, что статусы по 2 раза приходят

In [159]:
dfo_erka9 = dfo_erka9.merge(erka9, how='left', left_on='marketplace_number', right_on='er_order_id')

In [160]:
# есть еще 10 дублей, хз почему

In [161]:
dfo_erka9_mm = dfo_erka9.groupby('mm', as_index=False) \
                        .agg({'id': 'nunique', 
                           'order_id': 'nunique', 
                           'er_order_id': 'nunique'})

In [162]:
dfo_erka9_mm['percent'] = round(dfo_erka9_mm.order_id / dfo_erka9_mm.er_order_id * 100, 0)

In [163]:
dfo_erka9_mm

,mm,id,order_id,er_order_id,percent
0,АптекаФорте,782,526,609,86.0
1,Все аптеки,743,574,494,116.0
2,ОК Аптека,22,22,14,157.0
3,Озон RFBS,6728,5397,4937,109.0
4,Сбермегамаркет,894,644,593,109.0
5,Ютека,20783,17301,16277,106.0
6,ЯндексDBS,879,754,694,109.0


In [164]:
dfo_erka9_rk = dfo_erka9.groupby('rk', as_index=False).agg({'id': 'nunique', 
                                               'order_id': 'nunique', 
                                               'er_order_id': 'nunique'})
dfo_erka9_rk['percent'] = round(dfo_erka9_rk.order_id / dfo_erka9_rk.er_order_id * 100, 0)

In [165]:
dfo_erka9_rk

,rk,id,order_id,er_order_id,percent
0,"ООО ""ПУЛЬС Брянск""",473,424,403,105.0
1,"ООО ""ПУЛЬС Волгоград""",1376,1171,1147,102.0
2,"ООО ""ПУЛЬС Воронеж""",109,84,80,105.0
3,"ООО ""ПУЛЬС Екатеринбург""",123,101,93,109.0
4,"ООО ""ПУЛЬС Казань""",149,130,126,103.0
5,"ООО ""ПУЛЬС Краснодар""",632,507,481,105.0
6,"ООО ""ПУЛЬС Красноярск""",88,77,74,104.0
7,"ООО ""ПУЛЬС Новосибирск""",1539,1326,1219,109.0
8,"ООО ""ПУЛЬС СПб""",9765,8054,7616,106.0
9,"ООО ""ПУЛЬС Самара""",168,143,139,103.0


In [166]:
dfo_erka9_dd = dfo_erka9.groupby(dfo_erka9.created.dt.date) \
                        .agg({'id': 'nunique', 
                           'order_id': 'nunique', 
                           'er_order_id': 'nunique'})
dfo_erka9_dd['percent'] = round(dfo_erka9_dd.order_id / dfo_erka9_dd.er_order_id * 100, 0)

In [167]:
dfo_erka9_dd

,id,order_id,er_order_id,percent
created,,,,
2023-11-01,534,467,445,105.0
2023-11-02,833,712,682,104.0
2023-11-03,1113,927,867,107.0
2023-11-04,805,631,589,107.0
2023-11-05,863,721,684,105.0
2023-11-06,1094,902,857,105.0
2023-11-07,1284,1022,941,109.0
2023-11-08,1250,1096,1031,106.0
2023-11-09,1244,1064,1004,106.0


## Планета Здоровья 71

In [168]:
dfo_pz71 = dfo_pz.merge(dfs71, how='left', left_on='id', right_on='order_id')

In [169]:
dfo_pz71.shape

(135264, 14)

In [170]:
# есть 50 дублей, очевидно из-за того, что статусы по 2 раза приходят

In [171]:
dfo_pz71 = dfo_pz71.merge(pz71, how='left', left_on='guid', right_on='pz_order_guid')

In [172]:
# есть еще 10 дублей, хз почему

In [173]:
dfo_pz71_mm = dfo_pz71.groupby('mm', as_index=False) \
                        .agg({'id': 'nunique', 
                           'order_id': 'nunique', 
                           'pz_order_guid': 'nunique'})

In [174]:
dfo_pz71_mm['percent'] = round(dfo_pz71_mm.order_id / dfo_pz71_mm.pz_order_guid * 100, 0)

In [175]:
dfo_pz71_mm

,mm,id,order_id,pz_order_guid,percent
0,АСНА,2209,2196,1586,138.0
1,АналитФармация,2,2,0,inf
2,АптекаФорте,2258,2145,1580,136.0
3,В аптеке,241,239,134,178.0
4,Все аптеки,4266,4240,2980,142.0
5,ОК Аптека,27,24,5,480.0
6,Озон RFBS,18211,18100,13672,132.0
7,Планета Здоровья,47561,47519,0,inf
8,Сбермегамаркет,2455,2438,1702,143.0
9,ЦВА,837,833,568,147.0


In [176]:
dfo_pz71_rk = dfo_pz71.groupby('rk', as_index=False).agg({'id': 'nunique', 
                                               'order_id': 'nunique', 
                                               'pz_order_guid': 'nunique'})
dfo_pz71_rk['percent'] = round(dfo_pz71_rk.order_id / dfo_pz71_rk.pz_order_guid * 100, 0)

In [177]:
dfo_pz71_rk

,rk,id,order_id,pz_order_guid,percent
0,"ООО ""ПУЛЬС Брянск""",677,676,216,313.0
1,"ООО ""ПУЛЬС Волгоград""",523,523,0,inf
2,"ООО ""ПУЛЬС Воронеж""",1018,1016,0,inf
3,"ООО ""ПУЛЬС Екатеринбург""",33245,33223,10530,316.0
4,"ООО ""ПУЛЬС Иркутск""",90,90,66,136.0
5,"ООО ""ПУЛЬС Казань""",14098,14091,4305,327.0
6,"ООО ""ПУЛЬС Краснодар""",2287,2285,573,399.0
7,"ООО ""ПУЛЬС Новосибирск""",1200,1199,417,288.0
8,"ООО ""ПУЛЬС СПб""",5358,5347,2707,198.0
9,"ООО ""ПУЛЬС Самара""",917,877,0,inf


In [178]:
dfo_pz71_dd = dfo_pz71.groupby(dfo_pz71.created.dt.date) \
                        .agg({'id': 'nunique', 
                           'order_id': 'nunique', 
                           'pz_order_guid': 'nunique'})
dfo_pz71_dd['percent'] = round(dfo_pz71_dd.order_id / dfo_pz71_dd.pz_order_guid * 100, 0)

In [179]:
dfo_pz71_dd

,id,order_id,pz_order_guid,percent
created,,,,
2023-11-01,4710,4705,1,470500.0
2023-11-02,4897,4892,2,244600.0
2023-11-03,3847,3838,10,38380.0
2023-11-04,3377,3356,13,25815.0
2023-11-05,3839,3816,15,25440.0
2023-11-06,4656,4628,28,16529.0
2023-11-07,5337,5316,127,4186.0
2023-11-08,5415,5397,593,910.0
2023-11-09,4925,4905,3290,149.0


## Планета Здоровья 9

In [180]:
dfo_pz9 = dfo_pz.merge(dfs9, how='left', left_on='id', right_on='order_id')

In [181]:
dfo_pz9.shape

(134223, 14)

In [182]:
# есть 50 дублей, очевидно из-за того, что статусы по 2 раза приходят

In [183]:
dfo_pz9 = dfo_pz9.merge(pz9, how='left', left_on='guid', right_on='pz_order_guid')

In [184]:
# есть еще 10 дублей, хз почему

In [185]:
dfo_pz9_mm = dfo_pz9.groupby('mm', as_index=False) \
                        .agg({'id': 'nunique', 
                           'order_id': 'nunique', 
                           'pz_order_guid': 'nunique'})

In [186]:
dfo_pz9_mm['percent'] = round(dfo_pz9_mm.order_id / dfo_pz9_mm.pz_order_guid * 100, 0)

In [187]:
dfo_pz9_mm

,mm,id,order_id,pz_order_guid,percent
0,АСНА,2209,1881,1329,142.0
1,АналитФармация,2,2,0,inf
2,АптекаФорте,2258,1676,1556,108.0
3,В аптеке,241,238,124,192.0
4,Все аптеки,4266,3793,2615,145.0
5,ОК Аптека,27,26,4,650.0
6,Озон RFBS,18211,16228,12492,130.0
7,Планета Здоровья,47561,37260,0,inf
8,Сбермегамаркет,2455,2268,1710,133.0
9,ЦВА,837,677,458,148.0


In [188]:
dfo_pz9_rk = dfo_pz9.groupby('rk', as_index=False).agg({'id': 'nunique', 
                                               'order_id': 'nunique', 
                                               'pz_order_guid': 'nunique'})
dfo_pz9_rk['percent'] = round(dfo_pz9_rk.order_id / dfo_pz9_rk.pz_order_guid * 100, 0)

In [189]:
dfo_pz9_rk

,rk,id,order_id,pz_order_guid,percent
0,"ООО ""ПУЛЬС Брянск""",677,581,212,274.0
1,"ООО ""ПУЛЬС Волгоград""",523,419,0,inf
2,"ООО ""ПУЛЬС Воронеж""",1018,811,0,inf
3,"ООО ""ПУЛЬС Екатеринбург""",33245,28047,9906,283.0
4,"ООО ""ПУЛЬС Иркутск""",90,89,73,122.0
5,"ООО ""ПУЛЬС Казань""",14098,11912,4079,292.0
6,"ООО ""ПУЛЬС Краснодар""",2287,1835,515,356.0
7,"ООО ""ПУЛЬС Новосибирск""",1200,997,373,267.0
8,"ООО ""ПУЛЬС СПб""",5358,4628,2503,185.0
9,"ООО ""ПУЛЬС Самара""",917,689,0,inf


In [190]:
dfo_pz9_dd = dfo_pz9.groupby(dfo_pz9.created.dt.date) \
                        .agg({'id': 'nunique', 
                           'order_id': 'nunique', 
                           'pz_order_guid': 'nunique'})
dfo_pz9_dd['percent'] = round(dfo_pz9_dd.order_id / dfo_pz9_dd.pz_order_guid * 100, 0)

In [191]:
dfo_pz9_dd

,id,order_id,pz_order_guid,percent
created,,,,
2023-11-01,4710,4164,222,1876.0
2023-11-02,4897,4286,254,1687.0
2023-11-03,3847,3412,353,967.0
2023-11-04,3377,2908,351,828.0
2023-11-05,3839,3363,371,906.0
2023-11-06,4656,4103,511,803.0
2023-11-07,5337,4614,989,467.0
2023-11-08,5415,4748,1866,254.0
2023-11-09,4925,4333,2930,148.0


## Дальше можно не смотреть

In [ ]:
# unique_order_ids_df1 = erka['order_id'].unique()
# # Проверяем, содержатся ли эти уникальные значения во втором DataFrame
# mask = ~erka['order_id'].isin(df['marketplace_number'])

# # Используем маску для вывода уникальных order_id из df1, которых нет в df2
# missing_order_ids = erka.loc[mask, 'order_id'].unique()
# missing_order_ids

In [ ]:
dfo = pd.read_csv('https://redash.puls.ru/api/queries/8527/results.csv?api_key=04tKaxPAJfcexTsRwuWOfIFH7Xc1e8zAxoemdYja')
dfo.shape

In [ ]:
dfmp = dfmpp.merge(dfo, how='inner', left_on='order_id', right_on='id') \
        .loc[:, ['order_id', 'marketplace', 'rk', 'status', 'status_time']]  \
        .sort_values(by=['order_id', 'status_time'])

In [ ]:
dfmp

In [ ]:
# Создание столбца filtered_status (4 минут)
dfmp['filtered_status'] = dfmp.groupby('order_id')['status'].transform(lambda x: x.ne(x.shift()))

In [ ]:
dfmp = dfmp[dfmp['filtered_status'] == True]

In [ ]:
#dfmp['filtered_status'] = dfmp['status'].where(dfmp['filtered_status'] > 0, None)

In [ ]:
dfmp.status = dfmp.status.astype('str')

In [ ]:
# Группировка и агрегирование 
grouped_dfmp = dfmp.groupby(['order_id', 'marketplace', 'rk'], as_index=False).agg({'status': lambda x: ','.join(x)})

In [ ]:
#grouped_dfmp

In [ ]:
#grouped_dfmp[grouped_dfmp.status.str.contains(',81')]

In [ ]:
grouped_dfmp2 = grouped_dfmp.groupby(['status', 'marketplace', 'rk']).size().reset_index(name='num')

In [ ]:
grouped_dfmp2.num.sum()

In [ ]:
grouped_dfmp2

In [ ]:
# dfm = grouped_df.groupby('status').size().reset_index(name='num')
# dfm.sort_values('num', ascending=False).head(300)

In [ ]:
# dfm.columns = df.columns

In [ ]:
# # Преобразование строки статусов в список целых чисел
# dfm['status_sequence'] = dfm['status_sequence'].apply(lambda x: [int(s) for s in x.split(',')])

# # Создание списков для узлов и связей
# sources = []
# targets = []
# values = []

# # Наполнение списков значениями
# for _, row in dfm.iterrows():
#     sequence = row['status_sequence']
#     for i in range(len(sequence) - 1):
#         sources.append(sequence[i])
#         targets.append(sequence[i + 1])
#         values.append(row['num'])  # Вес связи равен 'num'       
    

In [ ]:
# # Создание и отображение диаграммы Санки
# fig = go.Figure(data=[go.Sankey(  arrangement = "snap",
#     node=dict(
#         pad=15,
#         thickness=20,
#         line=dict(color="black", width=0.5),
#         label=[str(i) for i in sorted(set(sources + targets))],  # Уникальные метки для узлов
#     ),
#     link=dict(  arrowlen=10,
#                 source=sources,  # Индексы источников
#                 target=targets,  # Индексы целей
#                 value=values     # Вес связей
#             )
# )])

# fig.update_layout(title_text="Sankey Diagram", font_size=10)
# fig.show()